This version is uses and saves all files as tiff images.

In [2]:
from PIL import Image
import numpy as np
import h5py
import matplotlib.pyplot as plt
import os
import fastmri
from fastmri.data import transforms as T
from fastmri.data.subsample import RandomMaskFunc, EquiSpacedMaskFunc


In [3]:
#my previous mask and recon functions

# Function to apply the custom mask
def apply_custom_mask(kspace, mask):
    return kspace * mask

# Function to reconstruct the image from k-space 
def reconstruct_image(kspace):
    image = np.fft.ifftshift(np.fft.ifftn(np.fft.fftshift(kspace), axes=(1, 2)))#add abs
    image = np.abs(image)
    final_image = np.sqrt(np.sum(image**2, axis=0))
    return np.abs(final_image)

# Function to normalize image data to the range [0, 255] for 8-bit grayscale images
def normalize_image(image):
    image = image - np.min(image)  
    image = image / np.max(image)  
    image = (image * 255).astype(np.uint8)  # Scale to [0, 255] and convert to 8-bit
    return image

def image_to_kspace(image):
    return np.fft.ifftshift(np.fft.fftn(np.fft.fftshift(image), axes=(0, 1)))

def create_random_mask(height, width, center_fraction=0.1, undersample_fraction=0.1, seed=50):
    """
    Create a 2D random mask with a fully sampled center region.

    Args:
        height: The height of the mask (frequency encoding dimension).
        width: The width of the mask (phase encoding dimension).
        center_fraction: Fraction of the image to be fully sampled at the center.
        undersample_fraction: Fraction of the remaining k-space to be sampled randomly.

    Returns:
        mask_2d: A binary mask with the same shape as the k-space data.
    """
    if seed is not None:
        np.random.seed(seed)  # Set the random seed locally
        mask = np.zeros((height, width))
    
    # Fully sample the center region
    center_height = int(height * center_fraction)
    center_width = int(width * center_fraction)
    center_start_h = (height - center_height) // 2
    center_start_w = (width - center_width) // 2
    mask[center_start_h:center_start_h + center_height, center_start_w:center_start_w + center_width] = 1
    
    # Randomly sample the remaining k-space
    remaining_mask = np.random.choice([0, 1], size=(height, width), p=[undersample_fraction, 1-undersample_fraction])
    mask = np.maximum(mask, remaining_mask)  # Ensure the center is fully sampled
    
    return mask

In [ ]:
# Path to your folder containing the HDF5 files
data_folder = r'D:\Class Project\209\brain_multicoil_train_batch_1'
file_names = [f for f in os.listdir(data_folder) if f.endswith('.h5')]

# Folders to save the images
output_folder_noisy = r'D:\Class Project\209\brain_multicoil_train_batch_1\noisy_images_0.1'
output_folder_ori = r'D:\Class Project\209\brain_multicoil_train_batch_1\ground_truth_images'

# Iterate through all the files
count = 0
total_files = len(file_names)
for file_name in file_names:
    file_path = os.path.join(data_folder, file_name)
    count += 1
    print(f"Processing: {count/total_files:.2%}")
    with h5py.File(file_path, 'r') as file:
        kspace = file['kspace'][:]  # Load all slices of multi-coil k-space data

        height, width = kspace.shape[2], kspace.shape[3]

        # Create the custom mask
        mask_2d = create_random_mask(height, width, center_fraction=0.2, undersample_fraction=0.1)

        for slice_idx in range(kspace.shape[0]-7):  # Iterate over slices
            # Extract k-space for the slice
            kspace_slice = kspace[slice_idx, :, :, :]
            
            # Reconstruct the multi-coil image using RSS
            reconstructed_image = reconstruct_image(kspace_slice)

            # Transform the reconstructed image back to k-space
            kspace_reconstructed = image_to_kspace(reconstructed_image)

            # Apply the mask to the k-space data
            masked_kspace = apply_custom_mask(kspace_reconstructed, mask_2d)

            # Transform the undersampled k-space back to the image domain
            undersampled_image = np.fft.ifft2(np.fft.ifftshift(masked_kspace))
            undersampled_image = np.abs(undersampled_image)

            # Normalize the images
            normalized_noisy_image = normalize_image(undersampled_image)
            normalized_image = normalize_image(reconstructed_image)

            # Convert the NumPy arrays to PIL Image objects
            tiff_image_noisy = Image.fromarray(normalized_noisy_image)
            tiff_image_ori = Image.fromarray(normalized_image)

            # Define the file names for saving
            noisy_img_file_name = f"{file_name}_noisy_slice_{slice_idx}.tiff"
            ori_img_file_name = f"{file_name}_ori_slice_{slice_idx}.tiff"

            # Save the noisy and original images in their respective folders
            tiff_image_noisy.save(os.path.join(output_folder_noisy, noisy_img_file_name))
            tiff_image_ori.save(os.path.join(output_folder_ori, ori_img_file_name))

Processing: 0.22%
Processing: 0.45%
Processing: 0.67%
Processing: 0.90%
Processing: 1.12%
Processing: 1.35%
Processing: 1.57%
Processing: 1.79%
Processing: 2.02%
Processing: 2.24%
Processing: 2.47%
Processing: 2.69%
Processing: 2.91%
Processing: 3.14%
Processing: 3.36%
Processing: 3.59%
Processing: 3.81%
Processing: 4.04%
Processing: 4.26%
Processing: 4.48%
Processing: 4.71%
Processing: 4.93%
Processing: 5.16%
Processing: 5.38%
Processing: 5.61%
Processing: 5.83%
Processing: 6.05%
Processing: 6.28%
Processing: 6.50%
Processing: 6.73%
Processing: 6.95%
Processing: 7.17%
Processing: 7.40%
Processing: 7.62%
Processing: 7.85%
Processing: 8.07%
Processing: 8.30%
Processing: 8.52%
Processing: 8.74%
Processing: 8.97%
Processing: 9.19%
Processing: 9.42%
Processing: 9.64%
Processing: 9.87%
Processing: 10.09%
Processing: 10.31%
Processing: 10.54%
Processing: 10.76%
Processing: 10.99%
Processing: 11.21%
Processing: 11.43%
Processing: 11.66%
Processing: 11.88%
Processing: 12.11%
Processing: 12.33%